# Milestone 1: Data Ingestion & Processing Layer

This notebook demonstrates the preprocessing of enterprise customer support data for knowledge graph readiness.

Steps performed:
- Data loading
- Cleaning
- Normalization
- Handling missing values
- Business enrichment
- Structured dataset generation

In [2]:
import pandas as pd

## Step 1: Load Raw Dataset

In [3]:
df = pd.read_excel("../data/raw/tickets.xlsx")
df.head()

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaT,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaT,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [4]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Ticket ID                     8469 non-null   int64         
 1   Customer Name                 8469 non-null   str           
 2   Customer Email                8469 non-null   str           
 3   Customer Age                  8469 non-null   int64         
 4   Customer Gender               8469 non-null   str           
 5   Product Purchased             8469 non-null   str           
 6   Date of Purchase              8469 non-null   datetime64[us]
 7   Ticket Type                   8469 non-null   str           
 8   Ticket Subject                8469 non-null   str           
 9   Ticket Description            8469 non-null   str           
 10  Ticket Status                 8469 non-null   str           
 11  Resolution                    2769 non-nu

## Step 2: Initial Data Inspection
Understanding missing values and dataset structure.

In [5]:
df.isnull().sum()

Ticket ID                          0
Customer Name                      0
Customer Email                     0
Customer Age                       0
Customer Gender                    0
Product Purchased                  0
Date of Purchase                   0
Ticket Type                        0
Ticket Subject                     0
Ticket Description                 0
Ticket Status                      0
Resolution                      5700
Ticket Priority                    0
Ticket Channel                     0
First Response Time             2819
Time to Resolution              5700
Customer Satisfaction Rating    5700
dtype: int64

In [6]:
df.columns

Index(['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age',
       'Customer Gender', 'Product Purchased', 'Date of Purchase',
       'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status',
       'Resolution', 'Ticket Priority', 'Ticket Channel',
       'First Response Time', 'Time to Resolution',
       'Customer Satisfaction Rating'],
      dtype='str')

In [9]:
df.duplicated().sum()

np.int64(0)

In [8]:
df['Ticket Status'].unique()

<StringArray>
['Pending Customer Response', 'Closed', 'Open']
Length: 3, dtype: str

In [10]:
df.isnull().sum().sort_values(ascending=False)

Customer Satisfaction Rating    5700
Resolution                      5700
Time to Resolution              5700
First Response Time             2819
Ticket ID                          0
Customer Name                      0
Customer Email                     0
Customer Age                       0
Customer Gender                    0
Ticket Subject                     0
Ticket Type                        0
Date of Purchase                   0
Product Purchased                  0
Ticket Priority                    0
Ticket Status                      0
Ticket Description                 0
Ticket Channel                     0
dtype: int64

In [11]:
df.shape

(8469, 17)

In [12]:
df['Ticket Status'].value_counts()

Ticket Status
Pending Customer Response    2881
Open                         2819
Closed                       2769
Name: count, dtype: int64

## Step 3: Data Cleaning
Standardizing text and formatting dates.

In [13]:
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [14]:
df = df.apply(lambda x: x.str.lower() if x.dtype == "object" else x)

In [15]:
df['Date of Purchase'] = pd.to_datetime(df['Date of Purchase'], errors='coerce')

## Step 4: Handling Missing Values
Missing values were interpreted as workflow states.

In [16]:
df['Resolution'] = df['Resolution'].fillna('not resolved yet')
df['Customer Satisfaction Rating'] = df['Customer Satisfaction Rating'].fillna('not rated')
df['Time to Resolution'] = df['Time to Resolution'].fillna('not available')
df['First Response Time'] = df['First Response Time'].fillna('not responded yet')

In [17]:
df['Ticket Priority'].value_counts()

Ticket Priority
Medium      2192
Critical    2129
High        2085
Low         2063
Name: count, dtype: int64

In [18]:
df['Ticket Channel'].value_counts()

Ticket Channel
Email           2143
Phone           2132
Social media    2121
Chat            2073
Name: count, dtype: int64

In [19]:
df['Ticket Channel'] = df['Ticket Channel'].replace({
    'Social media': 'Social Media'
})

In [20]:
df['Ticket Channel'].value_counts()

Ticket Channel
Email           2143
Phone           2132
Social Media    2121
Chat            2073
Name: count, dtype: int64

## Step 5: Data Enrichment
Creating business insight fields.

In [21]:
df['Ticket Status'].value_counts()

Ticket Status
Pending Customer Response    2881
Open                         2819
Closed                       2769
Name: count, dtype: int64

In [22]:
df['Customer Gender'].value_counts()

Customer Gender
Male      2896
Female    2887
Other     2686
Name: count, dtype: int64

In [23]:
df['Ticket State'] = df['Ticket Status'].replace({
    'Open': 'Active',
    'Pending Customer Response': 'Waiting',
    'Closed': 'Completed'
})
df['Ticket State'].value_counts()

Ticket State
Waiting      2881
Active       2819
Completed    2769
Name: count, dtype: int64

In [24]:
df['Resolution Status'] = df['Resolution'].apply(
    lambda x: 'Resolved' if x != 'not resolved yet' else 'Unresolved'
)
df['Resolution Status'].value_counts()

Resolution Status
Unresolved    5700
Resolved      2769
Name: count, dtype: int64

In [25]:
df['Severity'] = df['Ticket Priority'].replace({
    'Low': 'Minor',
    'Medium': 'Moderate',
    'High': 'Major',
    'Critical': 'Critical'
})
df['Severity'].value_counts()

Severity
Moderate    2192
Critical    2129
Major       2085
Minor       2063
Name: count, dtype: int64

In [26]:
df['Severity'].value_counts()

Severity
Moderate    2192
Critical    2129
Major       2085
Minor       2063
Name: count, dtype: int64

In [27]:
def categorize_time(x):
    if x == 'not available':
        return 'Unknown'
    elif isinstance(x, str):
        return 'Pending'
    else:
        return 'Resolved'

df['Resolution Time Category'] = df['Time to Resolution'].apply(categorize_time)
df['Resolution Time Category'].value_counts()

Resolution Time Category
Unknown     5700
Resolved    2769
Name: count, dtype: int64

## Step 6: Final Structured Dataset
Saving cleaned data for downstream usage.

In [28]:
df.to_excel("../data/processed/cleaned_tickets.xlsx", index=False)

## Outcome

The raw dataset has been transformed into a structured, normalized, and enriched enterprise-ready dataset.

This processed data is exposed via Flask API for downstream graph construction.